In [ ]:
from typing import TypedDict
from langgraph.graph import END, StateGraph, START
from langgraph.graph import MessagesState
from langchain.messages import HumanMessage, AIMessage


class MyState(MessagesState):
    val: int
    string: str
    test: str

def node_a(state: MyState):
    return {"val": state["val"]+1}

def node_b(state: MyState):
    return {"string": "Hi"}

def node_c(state: MyState):
    return {"messages": [HumanMessage(state["string"])]}

def node_d(state: MyState):
    return {"messages": [AIMessage("Hello")]}

workflow = StateGraph(MyState)

workflow.add_node("node_a", node_a)
workflow.add_node("node_b", node_b)
workflow.add_node("node_c", node_c)
workflow.add_node("node_d", node_d)

workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", "node_c")
workflow.add_edge("node_c", "node_d")
workflow.add_edge("node_d", END)

app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
app.invoke({"val": 1})

In [ ]:
temp_str = ""
if temp_str:
    print(1)
else:
    print(2)

print(HumanMessage(temp_str))

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import MessagesState

class State(MessagesState):
    question: Annotated[str, "User question"]
    generation: Annotated[str, "LLM generated answer"]
    documents: Annotated[list[str], "List of documents"]

In [ ]:
print(State.__annotations__)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage

LLM_ID = "Qwen/Qwen3-8B"
OPENAI_URL = "http://127.0.0.1:9805/v1"

llm = ChatOpenAI(
    api_key="EMPTY",
    base_url=OPENAI_URL,
    model=LLM_ID
)

result = llm.invoke([HumanMessage("Hwllo")])
print(result)

In [ ]:
from typing import Annotated
from langgraph.graph import MessagesState
from langchain_milvus import Milvus
from langchain_openai import OpenAIEmbeddings

class State(MessagesState):
    question: Annotated[str, "User question"]
    generation: Annotated[str, "LLM generated answer"]
    documents: Annotated[list[str], "List of documents"]

EMBED_MODEL_ID = "Qwen/Qwen3-Embedding-8B"
OPENAI_URL = "http://127.0.0.1:9804/v1"
MILVUS_URI = "http://127.0.0.1:19530"

embeddings = OpenAIEmbeddings(
    api_key="EMPTY",
    base_url=OPENAI_URL,
    model=EMBED_MODEL_ID
)

vector_store = Milvus(
    embedding_function=embeddings,
    collection_name="doc_embeddings",
    connection_args={
        "uri": MILVUS_URI,
        "token": "root:Milvus",
        "db_name": "doc_embeddings"
    },
    index_params={
        "index_type": "FLAT",
        "metric_type": "L2"
    },
)

def retrieve(state: State):
    question = state.get("question", "").strip()
    documents = vector_store.similarity_search(question, k=3)
    return {"documents": documents, "question": question}

In [ ]:
result = retrieve({"question": "스마트시티"})

In [ ]:
print(result)

In [ ]:
template = """You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. 
Your primary mission is to answer questions based on provided context or chat history.
Ensure your response is concise and directly addresses the question without any additional narration.

###

Your final answer should be written concisely (but include important numerical values, technical terms, jargon, and names), followed by the source of the information.

# Steps

1. Carefully read and understand the context provided.
2. Identify the key information related to the question within the context.
3. Formulate a concise answer based on the relevant information.
4. Ensure your final answer directly addresses the question.
5. List the source of the answer in bullet points, which must be a file name (with a page number) or URL from the context. Omit if the source cannot be found.

# Output Format:
[Your final answer here, with numerical values, technical terms, jargon, and names in their original language]

**Source**(Optional)
- (Source of the answer, must be a file name(with a page number) or URL from the context. Omit if you can't find the source of the answer.)
- (list more if there are multiple sources)
- ...

###

Remember:
- It's crucial to base your answer solely on the **PROVIDED CONTEXT**. 
- DO NOT use any external knowledge or information not present in the given materials.
- If you can't find the source of the answer, you should answer that you don't know.

###

# Here is the user's QUESTION that you should answer:
{question}

# Here is the CONTEXT that you should use to answer the question:
{context}

# Your final ANSWER to the user's QUESTION:"""

In [ ]:
def format_docs(docs):
    formatted = []
    for doc in docs:
        page = doc.metadata.get("page")
        page_str = f"<page>{page}</page>" if isinstance(page, int) else ""
        formatted.append(
            f"<document><content>{doc.page_content}</content>"
            f"<source>{doc.metadata.get('source', 'unknown')}</source>"
            f"{page_str}</document>"
        )
    return "\n\n".join(formatted)

In [ ]:
context = format_docs(result["documents"][:1])
print(context)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["question", "context"],
)

In [ ]:
rag_chain = prompt | llm

generation = rag_chain.invoke({"context": format_docs(result["documents"][:10]), "question": "스마트 시티 어떻게 되가나?"})

In [ ]:
print(generation)

In [ ]:
print(generation.content)

In [ ]:
a = [1,2,3,4,5]
a[:100]

In [ ]:
a = {}
type(a)